Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) #missing l2 regulartion and dropout

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.309323
Minibatch accuracy: 6.2%
Validation accuracy: 12.6%
Minibatch loss at step 50: 2.019864
Minibatch accuracy: 31.2%
Validation accuracy: 33.9%
Minibatch loss at step 100: 1.500088
Minibatch accuracy: 50.0%
Validation accuracy: 67.5%
Minibatch loss at step 150: 0.926062
Minibatch accuracy: 68.8%
Validation accuracy: 75.8%
Minibatch loss at step 200: 0.786045
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 250: 0.772515
Minibatch accuracy: 87.5%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.467629
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 350: 1.306695
Minibatch accuracy: 62.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.897861
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.548670
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.395972
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

conv_stride=1
pooling_stride=2
pooling_size=2


cnnWithPooling = tf.Graph()

with cnnWithPooling.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        conv = tf.nn.conv2d(pooling, layer2_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) #missing l2 regulartion and dropout

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 901

with tf.Session(graph=cnnWithPooling) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.137550
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.146665
Minibatch accuracy: 25.0%
Validation accuracy: 18.4%
Minibatch loss at step 100: 1.901022
Minibatch accuracy: 31.2%
Validation accuracy: 65.0%
Minibatch loss at step 150: 1.225215
Minibatch accuracy: 62.5%
Validation accuracy: 73.0%
Minibatch loss at step 200: 0.757439
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step 250: 0.831043
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.383286
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 350: 1.296713
Minibatch accuracy: 62.5%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.955948
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.480511
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 500: 0.323250
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### Adding Dropout

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

conv_stride=1
pooling_stride=2
pooling_size=2

keep_prob=0.75

cnnWithPoolingDropout = tf.Graph()

with cnnWithPoolingDropout.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        conv = tf.nn.conv2d(pooling, layer2_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        keep = tf.nn.dropout(hidden, keep_prob)

        return tf.matmul(keep, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) #missing l2 regulartion

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 1001

with tf.Session(graph=cnnWithPoolingDropout) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.958602
Minibatch accuracy: 0.0%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.859715
Minibatch accuracy: 25.0%
Validation accuracy: 22.9%
Minibatch loss at step 100: 2.043537
Minibatch accuracy: 25.0%
Validation accuracy: 50.7%
Minibatch loss at step 150: 1.181009
Minibatch accuracy: 56.2%
Validation accuracy: 65.6%
Minibatch loss at step 200: 0.900628
Minibatch accuracy: 81.2%
Validation accuracy: 69.1%
Minibatch loss at step 250: 0.883154
Minibatch accuracy: 75.0%
Validation accuracy: 72.9%
Minibatch loss at step 300: 0.531145
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 350: 1.306136
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Minibatch loss at step 400: 1.158736
Minibatch accuracy: 62.5%
Validation accuracy: 72.9%
Minibatch loss at step 450: 0.608600
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 500: 0.373222
Minibatch accuracy: 93.8%
Validation accuracy: 78.4%
Mi

### Adding L2

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

conv_stride=1
pooling_stride=2
pooling_size=2

keep_prob=0.75

C = 1e-3

cnnWithPoolingDropoutL2Reg = tf.Graph()

with cnnWithPoolingDropoutL2Reg.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        conv = tf.nn.conv2d(pooling, layer2_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        keep = tf.nn.dropout(hidden, keep_prob)

        return tf.matmul(keep, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
        C * (
            tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + 
            tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights)
            )
        )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 751

with tf.Session(graph=cnnWithPoolingDropoutL2Reg) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.256636
Minibatch accuracy: 18.8%
Validation accuracy: 9.6%
Minibatch loss at step 50: 2.523168
Minibatch accuracy: 6.2%
Validation accuracy: 12.3%
Minibatch loss at step 100: 2.396510
Minibatch accuracy: 12.5%
Validation accuracy: 19.6%
Minibatch loss at step 150: 1.870393
Minibatch accuracy: 50.0%
Validation accuracy: 43.1%
Minibatch loss at step 200: 1.415506
Minibatch accuracy: 56.2%
Validation accuracy: 56.6%
Minibatch loss at step 250: 1.181776
Minibatch accuracy: 68.8%
Validation accuracy: 68.7%
Minibatch loss at step 300: 0.701122
Minibatch accuracy: 81.2%
Validation accuracy: 72.8%
Minibatch loss at step 350: 1.770064
Minibatch accuracy: 50.0%
Validation accuracy: 69.6%
Minibatch loss at step 400: 1.409355
Minibatch accuracy: 62.5%
Validation accuracy: 76.1%
Minibatch loss at step 450: 0.990202
Minibatch accuracy: 81.2%
Validation accuracy: 76.7%
Minibatch loss at step 500: 0.690715
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Min

### Adding Decayed Learning rate

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

conv_stride=1
pooling_stride=2
pooling_size=2

keep_prob=0.75

cnnWithPoolingDropoutDecayedLearningRate = tf.Graph()

with cnnWithPoolingDropoutDecayedLearningRate.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        conv = tf.nn.conv2d(pooling, layer2_weights, [1, conv_stride, conv_stride, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(hidden, ksize=[1, pooling_size, pooling_size, 1], strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        keep = tf.nn.dropout(hidden, keep_prob)

        return tf.matmul(keep, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) #missing l2 regulartion

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.05, global_step=global_step, decay_steps=100, decay_rate=0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps = 1001

with tf.Session(graph=cnnWithPoolingDropoutDecayedLearningRate) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.003189
Minibatch accuracy: 0.0%
Validation accuracy: 10.6%
Test accuracy: 10.8%
Minibatch loss at step 50: 1.974791
Minibatch accuracy: 25.0%
Validation accuracy: 29.8%
Test accuracy: 31.8%
Minibatch loss at step 100: 2.094213
Minibatch accuracy: 25.0%
Validation accuracy: 51.8%
Test accuracy: 56.6%
Minibatch loss at step 150: 1.416928
Minibatch accuracy: 43.8%
Validation accuracy: 64.4%
Test accuracy: 70.2%
Minibatch loss at step 200: 1.044024
Minibatch accuracy: 62.5%
Validation accuracy: 64.1%
Test accuracy: 69.8%
Minibatch loss at step 250: 0.938157
Minibatch accuracy: 75.0%
Validation accuracy: 71.0%
Test accuracy: 77.4%
Minibatch loss at step 300: 0.530277
Minibatch accuracy: 87.5%
Validation accuracy: 74.4%
Test accuracy: 80.7%
Minibatch loss at step 350: 1.580624
Minibatch accuracy: 56.2%
Validation accuracy: 72.9%
Test accuracy: 79.0%
Minibatch loss at step 400: 1.236400
Minibatch accuracy: 50.0%
Validation accuracy: 77.0%
Test accuracy: